In [27]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [48]:
def get_soup (url):
    '''Fetch HTML of the page and parse HTML
    Return the parsed HTML'''
    headers = requests.utils.default_headers()

    headers.update(
        {
            'User-Agent': 'My User Agent 1.0',
        }
    )

    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")

    return soup

def get_price (property):
    '''Find the rental price of the property'''
    price_tag = property.find('p', class_='css-mgq8yx', attrs={'data-testid': 'listing-card-price'})

    # Extract the text content of the <p> tag
    price_text = price_tag.text

    # Use a regular expression to extract the numeric value from the price text
    price_match = re.search(r'\$\d[\d,]*\.?\d*', price_text)
    
    if price_match:
        price_number = price_match.group().replace('$', '').replace(',', '')
        return price_number
    else: 
        return None
    
def get_suburb (property):
    '''Find the suburb and postcode of the property'''
    address_line2 = property.find('span', {'data-testid': 'address-line2'})
    address_details = address_line2.find_all('span')

    if len(address_details) >= 3:
        suburb = address_details[0].text.strip()
        postcode = address_details[2].text.strip()

    return suburb, postcode

def standardize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove the plural 's' (if any) at the end of the word
    text = re.sub(r's$', '', text)
    return text

def get_features (property):
    # Initialize variables for bedrooms and bathrooms
    bedrooms = None
    bathrooms = None
    parkings = None

    # Extract bedrooms and bathrooms
    property_features = property.find_all('span', {'data-testid': 'property-features-text-container'})
    
    for feature in property_features:
        number = feature.contents[0].strip()
        if number == '−':
            number = 0

        label_tag = feature.find('span', {'data-testid': 'property-features-text'})
        if label_tag:
            label = label_tag.text.strip()
            label = standardize_text(label)
        else: continue

        if label == 'bed':
            bedrooms = number
        elif label == 'bath':
            bathrooms = number
        elif label == 'parking':
            parkings = number

    return bedrooms, bathrooms, parkings

def get_next_url (soup):
    page_link_tag = soup.find_all('a', {'data-testid': 'paginator-navigation-button'})
    for link_tag in page_link_tag:
        page_label_tag = link_tag.find('span', class_='css-16q9xmc')
        page_label = page_label_tag.text
        if page_label == 'prev page':
            continue  
        elif page_label == 'next page':
            url = "https://www.domain.com.au" + link_tag['href']
            return url
        else:
            return None


In [49]:
url = "https://www.domain.com.au/rent/vic/"
properties = []
count = 1

In [50]:
while url:
    print(count)
    soup = get_soup(url)
    listings = soup.find_all('div', {'class': 'css-qrqvvg'})
    for property in listings:
        price = get_price(property)
        if not price:
            continue
        suburb, postcode = get_suburb(property)
        bedrooms, bathrooms, parkings = get_features(property)
        property_type = property.find('span', class_='css-693528').text.strip()

        # Store the extracted data in a dictionary
        property_data = {
            'price (AUD per week)': price,
            'bedrooms': bedrooms,
            'bathrooms': bathrooms,
            'parkings': parkings,
            'property type': property_type,
            'suburb': suburb,
            'postcode': postcode
        }

        # Append the dictionary to the list of properties
        properties.append(property_data)
    url = get_next_url(soup)
    count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [51]:
# Convert list of dictionaries to a pandas DataFrame
df = pd.DataFrame(properties)

# Write the DataFrame to a CSV file
df.to_csv('properties.csv', index=False)

print('Data successfully written to properties.csv')

Data successfully written to properties.csv


## Aggregate data

In [ ]:
property_df = pd.read_csv('properties.csv')
mean_price_per_postcode = property_df.groupby('postcode')['price (AUD per week)'].mean()
mean_price_df = mean_price_per_postcode.reset_index()
# Save the DataFrame to a CSV file
mean_price_df.to_csv('mean_price_per_postcode.csv', index=False)

print("File saved successfully as 'mean_price_per_postcode.csv'")